# AEMET OpenData
Aemet pone a disposición de los interesados una interfaz amigable para hacer la descarga de todos los datos que oferta en OpenData (https://opendata.aemet.es/centrodedescargas/inicio).

En este notebook se muestra cómo se puede acceder a un subconjunto de los datos ofertados con Python, bien de modo interactivo o utilizando el __módulo aemet_open_data__. Para funcionar en Python tienes que haber instalado la librería Pandas.

Los datos que puedes decargar utilizando este notebook son:
1. Estaciones climatológicas con datos en Aemet OpenData. Aemet tiene muchas más estaciones con datos, pero la mayor parte no están en OpenData.
1. Variables climatológicas diarias.
1. Variables climatológicas mensuales.

Desde Aemet puedes descargar las variables climatológicas, pero se ponen limitaciones en el número de datos a descargar por petición. Por ejemplo, para los datos diarios hay que hacer una petición por estación y un máximo de 5 años de serie; para los datos mensuales, una estación y un máximo de 36 meses. Si quieres obtener datos de más de una estación para una serie larga tienes que repetir el proceso. En estos casos es cuando utilizar el módulo aemet_open_data u otro similar te permite ahorrar tiempo.

Un cierto inconveniente es que entre los datos de las estaciones climatológicas OpenData no figura el primer dato de cada serie. Si quieres obtener las series completas tendrás que empezar haciendo peticiones desde una fecha inicial antigua e ir moviéndo progresivamente el rango hasta encontrar el inicio, lo que es tedioso si no utilizas un proceso automatizado, y consumidor de recursos, pues en ocasiones tienes que hacer muchas peticiones que no van a devolver datos.  

Si no estás interesado en el modo interactivo y lo que quieres es disponer rápido de los datos, te recomiendo ir directamente al apartado de cómo se utiliza el módulo aemet_open_data.

En todos los casos necesitas una __clave de acceso__. Entra en la página de Aemet https://opendata.aemet.es/centrodedescargas/inicio y solicita una clave para operar en el servicio (APIKEY). La clave es una serie alfanumérica muy larga, sálvala en el fichero apikey.txt. Tu apikey no caduca.

## Python interactivo

Si tienes curiosidad de ver como se funciona de modo interactivo en Python.

Pasos:
1. Se hace una petición de datos a Aemet, para lo que se necesita formar una url específica de cada tipo de datos que queremos descargar.
1. Si la petición tiene éxito devuelve 2 url: una para descargar los datos y otra para descargar los metadatos. Estas 2 url tienen una tiempo de validez.


In [2]:
from datetime import date
import json
import pandas as pd
import requests

In [4]:
# Leo mi apikey
with open('apikey.txt') as f:
    myapikey = f.readline()

# Leo las urls que necesito (hay muchas más)
with open('urls.json', 'r') as f:
    urls = json.load(f)

querystring = {"api_key":myapikey}
headers = {'cache-control': "no-cache"}

# ver la url que teng preparadas y los parámetros (esta petición no tiene)aemet_open_data
urls.keys()

dict_keys(['estaciones', 'estacion_dia', 'estacion_mes'])

### Estaciones AEMET OpenData
Petición de todas las estaciones meteorológicas que sirven datos en el servicio OpenData 

In [5]:
# veo si la petición tiene paráetros (no los tiene)
urls['estaciones']

['https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones',
 []]

In [6]:
# veo la url sólo: 
urls['estaciones'][0]

'https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones'

In [7]:
# aquí hago la petición (request) y veo la respuesta (response y content)
url = urls['estaciones'][0]
response = requests.request("GET", url, headers=headers, params=querystring)
content = json.loads(response.content)
content

{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/5a60779b',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/0556af7a'}

In [10]:
# si la petición tiene éxito puedo descargar los datos; el formato original es json
# pero para majorar la visibilidad lo almaceno en una dataframe de pandas
df = pd.read_json(content["datos"], encoding='latin-1')
df.head()

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,08186,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,08181,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",,020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,08180,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,08174,015025E


In [11]:
# si la petición tuvo éxito también puedo descargar los metadatos
# en pantalla algunas columnas aparece truncadas, pero si las
# salvas a un fichero se graba el contenido completo
df_meta = pd.read_json(content["metadatos"], encoding='latin-1')
df_meta.head()

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'latitud', 'descripcion': 'latitud de l..."
1,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
2,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
3,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."
4,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'ubicación de ..."


In [12]:
# y almaceno los datos (y/o) los metadatos en un fichero (csv, Excel)
# en este caso guardo los datos como csv
# para guardar los metadatos cambia el nombre al fichero en dst y 
# sustituye df.to_csv por df_meta.to_csv 
dst = '.\download\estaciones.csv'
df.to_csv(dst, index=False)

### Datos climatológicos diarios de una estación y un rango de fechas de hasta 5 años
Si quieres más de una estación o un periodo de más de 5 años hay que repetir el proceso para cada estación y periodo de hasta 5 años

In [15]:
# vemos la url a la que se va a hacer la petición y los parámetros:
# 2 fechas en un formato específico y un código de estación idema
urls['estacion_dia']

['https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/fechaIniStr/fechafin/fechaFinStr/estacion/idema',
 ['fechaIniStr format AAAA-MM-DDTHH:MM:SSUTC',
  'fechaFinStr format AAAA-MM-DDTHH:MM:SSUTC',
  'idema']]

In [10]:
# Doy valor a una url que tengo preformada (url), defino los nombres de los parámetros (params)
# y asigno los valores de los parámetros
# Sólo puedo pedir los datos de una estación en un rango de fechas de menos de 5 años
# Si quiero más estaciones o un rango de fehas mayor hay que repetir el proceso cambiando
# el contenido de values
url = urls['estacion_dia'][0]
params = ('fechaIniStr', 'fechaFinStr', 'idema') 
values = ('2022-01-01T00:00:00UTC', '2023-01-01T23:59:59UTC', '7178I')

for par, value in zip(params, values):
    url = url.replace(par, value) 

print(url)

response = requests.request("GET", url, headers=headers, params=querystring)
response_json = response.json()
response_json

https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-01-01T00:00:00UTC/fechafin/2023-01-01T23:59:59UTC/estacion/7178I


{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/d73b5665',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}

In [11]:
# Igual que antes: si la respuesta tiene éxito puedo descargar los datos
df = pd.read_json(response_json["datos"], encoding='latin-1')
df.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-01-01,7178I,MURCIA,MURCIA,62,"13,2","0,0","2,9",07:40,"23,5",15:10,30.0,"0,8","3,9",08:10,"8,6","1022,7",Varias,"1019,2",15
1,2022-01-02,7178I,MURCIA,MURCIA,62,"13,0","0,0","3,5",07:10,"22,6",15:20,30.0,"1,1","3,6",22:20,"8,0","1022,7",10,"1018,9",15
2,2022-01-03,7178I,MURCIA,MURCIA,62,"12,2","0,0","2,1",07:10,"22,4",15:40,23.0,"0,8","4,2",19:30,"8,7","1020,5",00,"1013,9",24
3,2022-01-04,7178I,MURCIA,MURCIA,62,"13,8","0,0","4,3",04:10,"23,3",14:30,29.0,"1,4","10,8",20:30,"8,5","1013,9",00,"1002,5",16
4,2022-01-05,7178I,MURCIA,MURCIA,62,"13,2","0,0","10,1",23:59,"16,2",14:30,28.0,"4,7","15,8",02:20,"5,5","1011,8",10,"1006,6",02


In [12]:
# y los metadatos
df_meta = pd.read_json(response_json["metadatos"], encoding='latin-1')
df_meta.head()

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'fecha', 'descripcion': 'fecha del dia ..."
1,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
2,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'nombre (ubica..."
3,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
4,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."


In [29]:
# y salvar los datos y/o los metadatos a fichero (aquí en formato csv)
fo = '.\download\murcia_dia.csv'
df.to_csv(fo, index=False)

## Utilizando el módulo aemet_open_data
El módulo sirve para no tener que escribir tanto, sobre todo si quieres descargar datos de más de una estación y en periodos de más de 5 años para los datos diarios o 36 meses para datos mensuales.

Lo primero que hay que hacer son las importaciones:

In [1]:
# Lo más simple es tener el notebook en el directorio donde has guardado el fichero aemet_open_data.py
# hay otras técnnicas que no se cnsideran aquí
from datetime import date
import pandas as pd
from aemet_open_data import AemetOpenData
aod = AemetOpenData()

### Estaciones con datos de variables cimatológicas en OpenData
El método (función) sólo tiene un parámetro para indicar si quiero los datos o los metadatos

In [4]:
# metadatos
est_metadata = aod.climatologias_estaciones(metadata=True)

200, exito


In [5]:
# como la petición ha ido bien, veo los resultados
est_metadata

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'latitud', 'descripcion': 'latitud de l..."
1,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
2,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
3,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."
4,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'ubicación de ..."
5,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indsinop', 'descripcion': 'Indicativo ..."
6,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'longitud', 'descripcion': 'longitud de..."


In [6]:
# y si me interesa los salvo a fichero
fo = './download/estaciones_metadata.csv'
est_metadata.to_csv(fo, index=False)

In [7]:
# ahora los datos
est = aod.climatologias_estaciones(metadata=False)

200, exito


In [8]:
# veo unos pocos del principio
est.head()

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,08186,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,08181,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",,020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,08180,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,08174,015025E


In [10]:
# y los salvo a fichero
fo = './download/estaciones.csv'
est.to_csv(fo, index=False)

### Datos climatológicos diarios
Necesito definir la fecha inicial, la fecha final y la estación o estaciones de las que quiero los datos

El código de la estación lo puedes consultar de la consulta previa de estaciones. 

Aemet tiene muchas más estaciones, pero sólo algunas están activas en OpenData.

En la siguiente celda se definen los parámetros de la petición de datos. Internamente, una petición de datos puede ser resuelta con varias peticiones a Aemet; los resultados de cada una de estas peticiones internas a Aemet se graban en un fichero por si se produce un error inesperado no tener que empezar otra vez desde cero. 

In [2]:
# la fechas las defino con la functión date(año de 4 cifras, mes, día

d1 = date(1900, 1, 1)
d2 = date(2023, 1, 1)

# estas son las estaciones en el ejemplo, puedo añadir más
# si solo quiero una escribo, por ejemplo estaciones = '7178I' 
estaciones = ('6297', '7178I', '8019', '8025')

# nombre del directorio donde se irán grabando cada petición de datos a Aemet
# en el caso de que haya que hacer más de una petición, es mejor ir grabando
# los resultados de cada petición
dir_out = './download'

In [ ]:
# primero pido los matadatos si no los tengo ya de otra vez,
# ya que se devuelven muchos tipos de datos con un nombre corto
# te darás cuenta que los metadatos son genéricos del tipo de datos de la
# petición, pero esto funciona así, tienes que hacer una petición de datos
# bien formada y develve lo específico (los datos) y lo genérico (los metadatos)
# es por esto que no hace falta obtener los metadatos más de una vez
clima_dia_meta = aod.climatologias_diarias(d1, d2, estaciones, metadata=True)

In [17]:
# la visualización se trunca a partir de una cierta longitud
# pero al grabar se tiene el coontenido completo
clima_dia_meta.head()

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'fecha', 'descripcion': 'fecha del dia ..."
1,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
2,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'nombre (ubica..."
3,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
4,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."


In [18]:
# lo grabo en formato Excel
fo = './download/clima_dia_metadata.xlsx'
clima_dia_meta.to_excel(fo, index=False)

In [3]:
# datos diarios
clima_dia = aod.climatologias_diarias(d1, d2, estaciones, dir_out, False)

ReadTimeout: HTTPSConnectionPool(host='opendata.aemet.es', port=443): Read timed out. (read timeout=2)

Si te fijas en las respuestas que se van dando a las peticiones sucesivas, hay una que indica que No hay datos... Esto se debe a que la estación no tiene datos en las fechas indicadas en la misma línea. Hay algunas situaciones que pueden llevar a que el resultado de la petición no sea el esperado, por ejemplo si tecleamos mal el código de una estación.

In [19]:
# Grabo los datos, esta vez en formato csv
fo = './download/clima_dia.csv'
clima_dia.to_csv(fo, index=False)

### Datos climatológicos mensuales
Necesito definir un año inicial y un año final (los datos se descargan por años completos) y la estación o estaciones de las que quiero los datos.

En vez de años puedes facilitar fechas, pero de aquí lo que hace es extraer el año de cada fecha

El código de la estación lo puedes consultar de la consulta previa de estaciones. 

In [20]:
# la fechas las doy como año, pero podía hacerlo con las variables
# d1 y d2 que utilicé para descargar los datos diarios

y1 = 2000
y2 = 2023

# estas son las estaciones en el ejemplo, puedo añadir más
# si solo quiero una escribo, por ejemplo estaciones = '7178I' 
estaciones = ('7178I', '7002Y')

# hago directamente la petición de datos
clima_mes = aod.climatologias_mensuales(y1, y2, estaciones, metadata=False)

7178I, 2000, 2003: 200, exito
7178I, 2004, 2007: 200, exito
7178I, 2008, 2011: 200, exito
7178I, 2012, 2015: 200, exito
7178I, 2016, 2019: 200, exito
7178I, 2020, 2023: 200, exito
7002Y, 2000, 2003: 404, No hay datos que satisfagan esos criterios
7002Y, 2004, 2007: 404, No hay datos que satisfagan esos criterios
7002Y, 2008, 2011: 200, exito
7002Y, 2012, 2015: 200, exito
7002Y, 2016, 2019: 200, exito
7002Y, 2020, 2023: 200, exito


In [21]:
# veo unos pocos del principio
clima_mes.head()

,fecha,indicativo,p_max,n_cub,glo,hr,n_gra,n_fog,inso,q_max,...,w_med,nt_00,ti_max,n_nie,tm_mes,tm_max,nv_0100,ts_50,q_min,np_010
0,2000-1,7178I,8.6(10),3.0,28030.0,70.0,0.0,4.0,6.7,1025.6(31),...,4.0,3.0,9.6,0.0,9.0,14.8,0.0,10.4,1002.1(14),5.0
1,2000-2,7178I,0.0(--),0.0,40324.0,55.0,0.0,1.0,8.5,1029.8(05),...,7.0,0.0,17.0,0.0,14.4,21.6,0.0,14.1,1011.7(16),0.0
2,2000-3,7178I,2.4(13),4.0,51821.0,53.0,0.0,0.0,7.7,1025.2(08),...,8.0,0.0,16.8,0.0,14.9,21.7,0.0,16.3,999.8(28),1.0
3,2000-4,7178I,6.6(09),5.0,63227.0,43.0,0.0,1.0,8.8,1016.2(23),...,11.0,0.0,19.0,0.0,17.0,23.3,0.0,18.8,985.7(02),2.0
4,2000-5,7178I,13.7(05),4.0,70820.0,54.0,0.0,0.0,8.9,1014.6(31),...,10.0,0.0,19.6,0.0,21.3,27.1,0.0,22.4,1002.1(04),4.0


In [22]:
# como hay muchas columnas las intermedias no se muestran
# voy a grabar los datos a fichero, en este caso csv
fo = './download/clima_mes.csv'
clima_mes.to_csv(fo, index=False)

In [24]:
# como hay columnas que no sé lo que quieren decir voy a pedir los metadatos
clima_mes_meta = aod.climatologias_mensuales(y1, y2, estaciones, metadata=True)

7178I, 2000, 2003: 200, exito


In [25]:
# y los grabo directamente a fichero
fo = './download/clima_mes_meta.csv'
clima_mes_meta.to_csv(fo, index=False)

### Todo en uno
Si sabes las estaciones cuyos datos te interesan puedes descargar los datos diarios y mensuales directamente en una celda.

Pero antes ejecuta los import si no lo has hecho previamente (ver celda a continuación).

Y define los parámetros de la petición (rango de fechas y lista de estaciones) y los nombres de los ficheros donde se grabarán los resultados.

In [1]:
# celda opcional, por si no has hecho los import antes 
# (tampoco pasa nada si la ejecutas otra vez)
from datetime import date
import pandas as pd
from . import aemet_open_data/AemetOpenData
aod = AemetOpenData()

SyntaxError: invalid syntax (4263421571.py, line 5)

In [28]:
# supongo que ya tengo las estaciones de otra vez, me concentro en las series diarias y mensuales

# fecha inicial y final
d1 = date(2000, 1, 1)
d2 = date(2023, 1, 1)

# estaciones
estaciones = ('7178I', '7002Y')

# nombres de ficheros csv
fo_dia = './download/clima_dia.csv'
fo_mes = './download/clima_mes.csv'

# petición de datos diarios y grabo
clima_dia = aod.climatologias_diarias(d1, d2, estaciones, metadata=False)
if isinstance(clima_dia, pd.DataFrame):
    clima_dia.to_csv(fo_dia, index=False)

print('') # un espacio entre líneas
    
# petición de datos mensuales y grabo
clima_mes = aod.climatologias_mensuales(d1, d2, estaciones, metadata=False)
if isinstance(clima_mes, pd.DataFrame):
    clima_mes.to_csv(fo_mes, index=False)


7178I, 2000-01-01T00:00:00UTC, 2004-12-30T23:59:59UTC: 200, exito
7178I, 2004-12-31T00:00:00UTC, 2009-12-30T23:59:59UTC: 200, exito
7178I, 2009-12-31T00:00:00UTC, 2014-12-30T23:59:59UTC: 200, exito
7178I, 2014-12-31T00:00:00UTC, 2019-12-30T23:59:59UTC: 200, exito
7178I, 2019-12-31T00:00:00UTC, 2023-01-01T23:59:59UTC: 200, exito
7002Y, 2000-01-01T00:00:00UTC, 2004-12-30T23:59:59UTC: 404, No hay datos que satisfagan esos criterios
7002Y, 2004-12-31T00:00:00UTC, 2009-12-30T23:59:59UTC: 200, exito
7002Y, 2009-12-31T00:00:00UTC, 2014-12-30T23:59:59UTC: 200, exito
7002Y, 2014-12-31T00:00:00UTC, 2019-12-30T23:59:59UTC: 200, exito
7002Y, 2019-12-31T00:00:00UTC, 2023-01-01T23:59:59UTC: 200, exito

7178I, 2000, 2003: 200, exito
7178I, 2004, 2007: 200, exito
7178I, 2008, 2011: 200, exito
7178I, 2012, 2015: 200, exito
7178I, 2016, 2019: 200, exito
7178I, 2020, 2023: 200, exito
7002Y, 2000, 2003: 404, No hay datos que satisfagan esos criterios
7002Y, 2004, 2007: 404, No hay datos que satisfagan eso